In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import cv2
import supervision as sv
import os
from scipy.ndimage import distance_transform_edt
import numpy as np
from typing import Dict, List, Tuple
import pandas as pd 


In [3]:
def pixel_accuracy(pred, target):
    correct = (pred == target).sum()
    total = pred.size
    return correct / total

def intersection_over_union(pred, target):
    intersection = (pred & target).sum()
    union = (pred | target).sum()
    return intersection / union

def dice_coefficient(pred, target):
    intersection = (pred & target).sum()
    return 2 * intersection / (pred.sum() + target.sum())


def nsd_coefficient(pred, ref, tau = 1):
    # Get the surface (edges) of both segmentations
    pred_surface = pred - distance_transform_edt(1 - pred) > 0
    ref_surface = ref - distance_transform_edt(1 - ref) > 0

    # Calculate the distance transform for each segmentation
    dist_pred_to_ref = distance_transform_edt(1 - ref)
    dist_ref_to_pred = distance_transform_edt(1 - pred)

    # Check distances for each surface
    within_tau_pred = dist_pred_to_ref[pred_surface] <= tau
    within_tau_ref = dist_ref_to_pred[ref_surface] <= tau

    # Compute the NSD
    nsd_pred = np.sum(within_tau_pred) / np.sum(pred_surface)
    nsd_ref = np.sum(within_tau_ref) / np.sum(ref_surface)
    
    nsd = 0.5 * (nsd_pred + nsd_ref)

    return nsd

def calculate_metrics(pred_list, target_list, metrics):
    if metrics is None:
        metrics = ['pixel_accuracy', 'iou', 'dice', 'nsd']
        
    results = {metric: 0 for metric in metrics}
    n = len(pred_list)  # Assuming pred_list and target_list are of the same length

    for pred, target in zip(pred_list, target_list):
        if 'pixel_accuracy' in metrics:
            results['pixel_accuracy'] += pixel_accuracy(pred, target)
        if 'iou' in metrics:
            results['iou'] += intersection_over_union(pred, target)
        if 'dice' in metrics:
            results['dice'] += dice_coefficient(pred, target)
        if 'nsd' in metrics:
            results['nsd'] += nsd_coefficient(pred, target)

    # Calculate the mean for each metric
    for metric in results:
        results[metric] /= n

    return results


def get_segmentation_mask_from_image(img, shape=(1920, 1080)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    masks = img > 0
    return masks
    




In [4]:

def load_and_process_image(mask_path: str, original_mask_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Load and preprocess mask images."""
    mask = cv2.imread(mask_path)
    original_mask_img = cv2.imread(original_mask_path)
    
    # Resize logic
    if original_mask_img.shape[:2] != mask.shape[:2]:
        target_shape = (
            min(original_mask_img.shape[1], mask.shape[1]),
            min(original_mask_img.shape[0], mask.shape[0])
        )
        original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])
        mask = cv2.resize(mask, target_shape[::-1])
    
    return get_segmentation_mask_from_image(mask, original_mask_img.shape), \
           get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)




In [12]:

def collect_metrics(sessoes_folder: str, original_mask_folder: str) -> Tuple[pd.DataFrame, Dict]:
    """Collect metrics for all images and organize them into a DataFrame."""
    # Initialize data storage
    data_rows = []
    groups_data = {
        "generated": {f"group_{i}": {"metrics": [], "time": []} for i in range(1, 5)}
    }
    
    print(f"Processing folders:")
    print(f"Total folders: {len(os.listdir(sessoes_folder))}")
    print(f"SAM folders: {len([f for f in os.listdir(sessoes_folder) if 'sam' in f])}")
    print(f"Manual folders: {len([f for f in os.listdir(sessoes_folder) if 'manual' in f])}\n")

    for sessao in os.listdir(sessoes_folder):
        # Get group name
        group_name = next((g for g in ["group_1", "group_2", "group_3", "group_4"] if g in sessao), None)
        if not group_name:
            continue

        masked_path = os.path.join(sessoes_folder, sessao)

        for image in os.listdir(masked_path): 
            original_mask_name = image.replace('.jpg','_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)

            if not os.path.exists(original_mask_path):
                continue

            mask, original_mask = load_and_process_image(os.path.join(masked_path,
                                                                      image),
                                                         original_mask_path)
            
            pixel_acc, iou, dice, nsd = calculate_metrics([mask], [original_mask], None).values()

            data_rows.append({
                "group": group_name,
                "image": image,
                "pixel_acc": pixel_acc,
                "iou": iou,
                "dice": dice,
                "nsd": nsd, 
                "time": None
            })


            groups_data["generated"][group_name]["metrics"].append({
                "pixel_acc": pixel_acc,
                "iou": iou,
                "dice": dice,
                "nsd": nsd
            })
            groups_data["generated"][group_name]["time"].append(None)

    return pd.DataFrame(data_rows), groups_data

In [10]:
sessoes_folder = "./sam_generated"
original_mask_folder = 'pad_segmentation_all/all-mask'

In [13]:
df, group_data = collect_metrics(sessoes_folder, original_mask_folder)

df.to_csv("metrics_sam_generated.csv", index=False)

Processing folders:
Total folders: 4
SAM folders: 4
Manual folders: 0

